In [1]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
import ctypes
import multiprocessing
from multiprocessing import Process, Manager, cpu_count

In [2]:
## 读取原始csv
data_raw = pd.read_csv("/home/anichikage/workspace/dram/dram-failure-prediction/data/raw/ali/mcelog.csv")
print(data_raw)

                   sid  memoryid  rankid  bankid     row  col  error_type  \
0             Server_1        10       1      13    3334  808           2   
1             Server_1        10       1      13    3334  808           2   
2             Server_1        10       1      13    3334  808           2   
3             Server_1        10       1      13    3334  808           2   
4             Server_1        10       1      13    3334  808           2   
...                ...       ...     ...     ...     ...  ...         ...   
75214687  Server_30500        16       0       8   51199  576           2   
75214688  Server_30501         0       0      12  109213  592           2   
75214689  Server_30502        20       1       7  120731  624           2   
75214690  Server_30502        20       1       7  120731  624           2   
75214691  Server_30502        20       1       7  120731  624           2   

                   error_time  
0         0001-06-14 06:40:59  
1         0

In [3]:
grouped = data_raw.groupby('sid') # 按sid将Excel分组
print(len(grouped))

30502


In [4]:
## 获取第一个分组
group_one = grouped.get_group("Server_40")
print(group_one)

             sid  memoryid  rankid  bankid    row  col  error_type  \
23506  Server_40        20       1       9  71065  880           2   
23507  Server_40        20       1       9  71065  880           2   
23508  Server_40        20       1       9  71065  880           2   
23509  Server_40        20       1       9  71065  880           2   
23510  Server_40        20       1       9  71065  880           2   
...          ...       ...     ...     ...    ...  ...         ...   
23747  Server_40        20       1       9  71065  880           2   
23748  Server_40        20       1       9  71065  880           2   
23749  Server_40        20       1       9  71065  880           2   
23750  Server_40        20       1       9  71065  880           2   
23751  Server_40        20       1       9  71065  880           2   

                error_time  
23506  0001-01-01 06:01:58  
23507  0001-01-02 05:34:37  
23508  0001-01-03 05:07:16  
23509  0001-01-04 04:39:55  
23510  0001-01

In [5]:
# 原数据中最大的bankid row col
max_bank = 16 # 0~15
max_row = 131072 # 0~131071
max_col = 1017 # 0~1016

In [6]:
rows_per_block = 4096  # 行压缩块大小为 4096
cols_per_block = 64    # 列压缩块大小为 64

In [7]:
## 将bank,row ,col存入一个大矩阵，再压缩成16x32x16
def process_group(group_one):
    matrix_one = np.zeros((max_bank, max_row, max_col))
    for _ , pos in group_one.iterrows():
        bank_index = pos["bankid"]
        row_index = pos["row"]
        col_index = pos["col"]

        matrix_one[bank_index, row_index, col_index] += 1 # 16x131071x1017 的大矩阵

    # 压缩成16x32x16尺寸   
    matrix_small = np.add.reduceat(
    np.add.reduceat(matrix_one, np.arange(0, matrix_one.shape[1], rows_per_block), axis=1),# 起点、终点、步长
    np.arange(0, matrix_one.shape[2], cols_per_block), axis=2)
    return matrix_small

In [8]:
# 循环处理函数
def process_group_3(grouped, idx):
    matrix_small = process_group(grouped.get_group(f"Server_{idx+1}"))
    shared_np_arr[idx] = matrix_small

In [9]:
# 创建共享数组 
array_shape = (30502, 16,32, 16)  # 30502个数组，每个数组大小为16x32x16
# 创建一个共享数组
shared_arr_base = multiprocessing.Array(ctypes.c_double, int(np.prod(array_shape)))
# 将共享数组转换为 NumPy 
shared_np_arr = np.frombuffer(shared_arr_base.get_obj(), dtype=np.float64).reshape(array_shape)


In [ ]:
print(shared_np_arr[39,9,:,:]) # 确认这个数组是不是存对了

In [10]:
# 创建30个进程，每次处理1017个样本

# 同一个进程，同时能打开的文件数是有限的，一般是一千多，当时每次都是运行到2013就报错了
for idx in range(30): # tqdm是显示进度条
    processes = []
    for j in tqdm(range(idx*1017, (idx+1)*1017)): 
        p = Process(target = process_group_3, args = (grouped, j)) # 创建一个多进程任务
        processes.append(p) # 将进程添加到列表
        p.start() # 启动进程，开始执行process_one_agg(idx)
    for p in processes:
        p.join() # 等待进程完成

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 1011/1017 [23:35<00:08,  1.43s/it]Process Process-30504:
Traceback (most recent call last):
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 1012/1017 [23:36<00:07,  1.42s/it]  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Process Process-30505:
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 1013/1017 [23:38<00:05,  1.40s/it]  File "/tmp/ipykernel_1550801/1826231526.py", line 3, in process_group_3
    matr

In [20]:
group_one = grouped.get_group("Server_30440")
print(group_one)

                   sid  memoryid  rankid  bankid     row  col  error_type  \
75188529  Server_30440         4       1       8  110386  376           2   
75188530  Server_30440         4       1       8  110386  376           2   
75188531  Server_30440         4       1       8  110386  320           2   
75188532  Server_30440         4       1       8  110386  312           2   
75188533  Server_30440         4       1       8  110386  296           2   
...                ...       ...     ...     ...     ...  ...         ...   
75188723  Server_30440         4       1       8  110386    0           2   
75188724  Server_30440         4       1       8  110386   16           2   
75188725  Server_30440         4       1       8  110386    8           2   
75188726  Server_30440         4       1       8  110386   16           2   
75188727  Server_30440         4       1       8  110386    0           2   

                   error_time  
75188529  0001-07-16 11:52:09  
75188530  0

In [23]:
print(shared_np_arr[30439,6,:,:])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [24]:
data_mce3d = shared_np_arr

In [28]:
print(data_mce3d.shape)

(30502, 16, 32, 16)


In [29]:
np.save('data_processed/data_mce3d.npy', data_mce3d) 

In [30]:
# 归一化
data_mce = np.load("data_processed/data_mce3d.npy")

In [31]:
data_norm = 2 / (1 + np.exp(-data_mce)) - 1

In [ ]:
print(data_norm[0,13,:,:])

In [33]:
np.save('data_processed/data_norm.npy', data_norm)

In [ ]:
print(sum(data_norm[30501,:,:,:]))